In [1]:
import pandas as pd
import plotly.express as px
from scipy.optimize import brentq
from datetime import date, timedelta
import math

In [2]:
def init_cashflow(drawdown, n, start_date):
  d = start_date.day
  start_date -= timedelta(days=d)
  ix = pd.date_range(start=start_date, periods=n+1 , freq='MS')
  ix = ix + timedelta(days=d-1)
  days = ix.to_series().diff().dt.days.fillna(0).astype(int)
  df = pd.DataFrame({'days' : days, 'capital' : float(drawdown), 'repayment' : 0., 'interest': 0., 'int_paid':0., 'frac_int':0., 'outstanding':0.}, index=ix)
  df.iat[0,6] = df.iat[0,1]
  return df

def evaluate_cashflow(df, repayment, interest_rate):
  '''updates df and return the final outstanding amount given a fixed repayment amount'''
  df.iloc[1:,2] = repayment
  prev = None
  for i in df.index:
    # copy prev outstanding to current capital
    if prev:
      df.at[i,'capital'] = df.at[prev,'outstanding']
    # calculate row
    carried_interest = df.at[prev,'frac_int'] if prev else 0.
    df.at[i,'interest'] = df.at[i,'days'] * df.at[i,'capital'] * interest_rate / 365 + carried_interest
    df.at[i,'int_paid'] = math.floor(df.at[i,'interest'] * 100) / 100
    df.at[i,'frac_int'] = df.at[i,'interest'] - df.at[i,'int_paid']
    df.at[i,'outstanding'] = df.at[i,'capital'] - df.at[i,'repayment'] + df.at[i,'int_paid']
    prev = i
  return df.iat[-1,6]

def solve_cashflow(df, interest_rate):
  ubound = df.iat[0,1]
  return brentq(lambda x: evaluate_cashflow(df, x, interest_rate), 0, ubound)

def example_cashflow(drawdown: float, interest_rate: float, months: int):
    df = init_cashflow(drawdown, months, date.today())
    r = solve_cashflow(df, interest_rate)
    # re-evaluate with rounded repayment
    evaluate_cashflow(df, round(r,2), interest_rate)
    print('--- example cashflow ---')
    print(f'drawdown {drawdown:.2f}, interest rate {interest_rate * 100:.4f}%, monthly payment {round(r,2):.2f}')
    print(f'total paid {df["repayment"].sum():.2f} , total interest {df["int_paid"].sum():.2f}, total frac {df["frac_int"].sum()}')
    return df

In [3]:
example_cashflow(1200, 0.06, 12)

--- example cashflow ---
drawdown 1200.00, interest rate 6.0000%, monthly payment 103.27
total paid 1239.24 , total interest 39.26, total frac 0.052878356164431595


,days,capital,repayment,interest,int_paid,frac_int,outstanding
2024-09-19,0,1200.00,0.00,0.000000,0.00,0.000000,1200.00
2024-10-19,30,1200.00,103.27,5.917808,5.91,0.007808,1102.64
2024-11-19,31,1102.64,103.27,5.626741,5.62,0.006741,1004.99
2024-12-19,30,1004.99,103.27,4.962856,4.96,0.002856,906.68
2025-01-19,31,906.68,103.27,4.623198,4.62,0.003198,808.03
2025-02-19,31,808.03,103.27,4.120830,4.12,0.000830,708.88
2025-03-19,28,708.88,103.27,3.263620,3.26,0.003620,608.87
2025-04-19,31,608.87,103.27,3.106355,3.10,0.006355,508.70
2025-05-19,30,508.70,103.27,2.515013,2.51,0.005013,407.94
2025-06-19,31,407.94,103.27,2.083830,2.08,0.003830,306.75
